In [1]:
#Import libraries

import gzip #for parsing gz files
import pandas as pd
import re 
import inflect #numeric/cardinal

In [86]:
dirPath = r"C:\Study\Projects\NLP\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"
qaPath = r"C:\Study\Projects\NLP\data" 
path = dirPath + "\\" + reviewPath

In [87]:
dataFile = gzip.open(path, 'r')

In [88]:
# Need this function because large amount of data we are going to load up

# http://jmcauley.ucsd.edu/data/amazon/index.html
# eval function allows text to be parsed as python code
# parse function is a generator with yield
# yield when we want to iterate over a sequence, but don't want to store the entire sequence in memory
def parse(dataFile):
    for each in dataFile:
        yield eval(each)

In [89]:
# for each review for a product in dataFile maps dict[nth] = review (nth)
# enumerate can't be used on a generator
count = 0
productReviews = {}
for eachReview in parse(dataFile):
    productReviews[count] = eachReview
    count = count + 1


In [90]:
len(productReviews)

134476

In [213]:
productReviews[101718]

{'reviewerID': 'A1W7J7X96MP32U',
 'asin': 'B004C29AFS',
 'reviewerName': 'Rocco',
 'helpful': [3, 3],
 'reviewText': "I rarely give negative reviews but this think just doesn't last long.  I even bought a second one to verify I just didn't get a bad one, and six months later and it's broken...  Again.  It just stops turning on.  The battery holder is weak and do much as touch the case and the unit shuts off.  I really wanted this product to work for me but after the 2nd one going bad I have to give upI decided to open it up and see what the problem was.  The solder connections look like a 3 year old did it. Cheap flimsy wire and secured poorly",
 'overall': 1.0,
 'summary': 'This is a peice of junk',
 'unixReviewTime': 1365465600,
 'reviewTime': '04 9, 2013'}

In [95]:
productsReviewData = pd.DataFrame.from_dict(productReviews, orient='index')

In [92]:
productsReviewData.shape
productsReviewData.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [93]:
reduceData = productsReviewData.loc[:, ['asin', 'reviewerName', 'reviewText', 'helpful', 'overall']]




In [94]:
rating = lambda x: 1 if x > 2.0 else 0

In [210]:
reduceData['rating'] = reduceData['overall'].apply(rating)
unfavorableReviews = list(reduceData.loc[ reduceData['rating'] == 0, 
                                         ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall').index)
#unfavorableReviews
reduceData.loc[ reduceData['rating'] == 0, ['reviewText', 'helpful', 'overall', 'rating']].sort_values(by='overall')


,reviewText,helpful,overall,rating
134322,Amazing that they designed this thing without ...,"[2, 2]",1.0,0
55236,I was ready to pick up one of these at a woodw...,"[30, 38]",1.0,0
55234,I bought this set from amazon but is not like ...,"[8, 15]",1.0,0
105852,Horrible.Not one blade runs true. Some are as...,"[0, 0]",1.0,0
55216,This product is almost a joke. The case is to...,"[4, 9]",1.0,0
55201,Well I came back to change my review of the pr...,"[0, 0]",1.0,0
55196,"Supposedly, these bits slice thru stainless st...","[2, 5]",1.0,0
105862,I bought this because I need to cut a vent hol...,"[0, 0]",1.0,0
55144,I have had this on my door for quite a while a...,"[0, 0]",1.0,0
55268,Appeared to be a 'perfect' light bar to sit ab...,"[0, 1]",1.0,0


In [97]:
reviewTextProcessing = pd.DataFrame(reduceData.reviewText.copy())

In [98]:
reviewTextProcessing['rating'] = reduceData['rating']
reviewTextProcessing['overall'] = reduceData['overall']

In [99]:
productsReviewData.dtypes
len(productsReviewData)
# reviewText, rating, overall
reviewTextProcessing.shape

(134476, 3)

In [100]:
#reviewTextProcessing.loc[reviewTextProcessing['rating']==0, ['rating', 'overall', 'Processed', 'SentimentModel']]

In [101]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import Contractions_Words as contractions
import importlib
importlib.reload(contractions)

contractions_dict = contractions.contractions_dict

print(contractions_dict.keys())
#print(contractions_dict["s***"])

pattern = '%s' % '|'.join(contractions_dict.keys())
#pattern = r"%s" % pattern
#print(pattern)
contractions_re = re.compile(pattern)
def expand_contractions(s, contractions_dict=contractions_dict):
    #edit for special case like s*** etc
    s = re.sub("\*", "#", s)
    def replace(match):
        #print(type(match), match)
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)



dict_keys(["ain't", "aren't", "can't", "can't've", "'cause", "could've", "couldn't", "couldn't've", "didn't", "doesn't", "don't", "hadn't", "hadn't've", "hasn't", "haven't", "he'd", "he'd've", "he'll", "he'll've", "he's", "how'd", "how'd'y", "how'll", "how's", "I'd", "I'd've", "I'll", "I'll've", "I'm", "I've", "isn't", "it'd", "it'd've", "it'll", "it'll've", "it's", "let's", "ma'am", "mayn't", "might've", "mightn't", "mightn't've", "must've", "mustn't", "mustn't've", "needn't", "needn't've", "o'clock", "oughtn't", "oughtn't've", "shan't", "sha'n't", "shan't've", "she'd", "she'd've", "she'll", "she'll've", "she's", 's###', "should've", "shouldn't", "shouldn't've", "so've", "so's", "that'd", "that'd've", "that's", "there'd", "there'd've", "there's", "they'd", "they'd've", "they'll", "they'll've", "they're", "they've", "to've", "wasn't", "we'd", "we'd've", "we'll", "we'll've", "we're", "we've", "weren't", "what'll", "what'll've", "what're", "what's", "what've", "when's", "when've", "where

In [102]:
sample1 = "s***"
print(re.sub("\*", "#", sample1))

sample = "I don\'t know what to say nay. I haven\'t. s###"
expand_contractions(sample)
expand_contractions(sample1)

s###


'shit'

In [103]:
from nltk.tokenize import word_tokenize, sent_tokenize
import string

# return both wholeText, paragraph Text
def singleReviewCleanUp(sample):
    #print(sample)
    #sample = sample.decode("utf-8")
    sample = expand_contractions(sample)
    sample = sample.lower()
    
    #for word in sample:
    #    print(lemmatizer.lemmatize(word, wordnet.VERB))
    
    review = sent_tokenize(sample)
    #print(review)
    for count, eachSent in enumerate(review):
        #remove numbers
        eachSent = re.sub("r'\d+/\d+|\d+|\$\d+.\d+", " ", eachSent)
        #remove string
        eachSent = re.sub('[%s]' % re.escape(string.punctuation), ' ', eachSent)
        #word tokens
        #print([lemmatizer.lemmatize(text, wordnet.VERB) for text in eachSent.split()])
        eachSent = word_tokenize(eachSent)
        review[count] = eachSent
        # lemmatize and remove stopWords
        #eachSent = [word for word in eachSent if not word in stopWords]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent]
        #eachSent = [lemmatizer.lemmatize(word, wordnet.VERB) for word in eachSent if not word in stopWords]
        review[count] = " ".join(eachSent)
    wholeText = [" ".join(review) ]
    paragraphText = review
    #return wholeText
    #return paragraphText
    return paragraphText, wholeText

#paragraphText, wholeText = singleReviewCleanUp(sample)
#wholeText, paragraphText
#paragraphText

In [104]:
#Give me back text in a paragraph sentence forms. They do have more values than a clobbed of text based on Polarity Check.
cleanUpTextParagraph = lambda text: singleReviewCleanUp(text)[0]


In [105]:
%%time
reviewTextProcessing['Processed'] = reviewTextProcessing.reviewText.apply(cleanUpTextParagraph)


Wall time: 3min 26s


In [21]:
#sample = " I don't know what to do. I need to test through tough. Do you have any suggestions"
#result = cleanUpTextParagraph(sample)
#result

In [22]:
from textblob import TextBlob

# need to do a data check first
def getPolaritySubjectivity(data):
    #print(data)
    paragraphText = data
    sumPolarity = []
    sumSubjectivity = []
    for each in paragraphText:
        sumPolarity.append(TextBlob(each).sentiment.polarity)
        sumSubjectivity.append(TextBlob(each).sentiment.subjectivity)
    #print("sumPol", sumPolarity)
    return sumPolarity, sumSubjectivity
    #return sum(sumPolarity), sum(sumSubjectivity)

    #print(each, TextBlob(each).sentiment.polarity, TextBlob(each).sentiment.subjectivity)
#(sample2, TextBlob(sample2).sentiment.polarity, TextBlob(sample2).sentiment.subjectivity)


In [152]:
#Return the sume of polarity in a paragraph sentence when give a text in a list form of sentence

getPolarityFromText = lambda text: getPolaritySubjectivity(text)[0]

In [153]:
#for each in sampleText:
#    sample = cleanUpTextParagraph(each)
#    print(sample, getPolarityFromText(sample))

In [154]:
#print(stopWords)

In [155]:
#sentiment = lambda text: TextBlob(text).sentiment.polarity

In [156]:
adjustSentimentModel = lambda x: [sum(x), 0.0] if sum(x) < 0 else [sum(x), 1.0]

In [157]:
%%time
reviewTextProcessing['Sentiment']= reviewTextProcessing.Processed.apply(getPolarityFromText)

Wall time: 7min


In [158]:
%%time
reviewTextProcessing['SentimentModel']= reviewTextProcessing.Sentiment.apply(adjustSentimentModel)

Wall time: 115 ms


In [30]:
#reviewTextProcessing.loc[unfavorableReviews, ['Processed', 'Sentiment', 'SentimentModel']]
#reviewTextProcessing.dtypes

In [160]:
reviewTextProcessing['CleanedJoin'] = reviewTextProcessing['Processed'].apply(', '.join)


In [161]:
reviewTextProcessing.shape
reviewTextProcessing.columns

Index(['reviewText', 'rating', 'overall', 'Processed', 'Sentiment',
       'SentimentModel', 'CleanedJoin'],
      dtype='object')

In [162]:
#from gensim import corpora
#dictionary = corpora.Dictionary(T)
#print(dictionary.token2id)

In [163]:
reviewTextProcessing['CleanedJoin'].head(5)

0    i hate it when my shirt collars not otherwise ...
1    these little magnets are really powerful for t...
2    i wanted something this small to mount on the ...
3    i use these to magnetize my warhammer k miniat...
4    they are soo freaking annoying, why, you spend...
Name: CleanedJoin, dtype: object

In [164]:

import itertools

largeText = reviewTextProcessing.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

134476

In [165]:
sentiment = reviewTextProcessing['SentimentModel'].to_list()

In [166]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

num_words = 10000


In [167]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [168]:

size = int(len(largeText)*.75)
x_train_text = largeText[:size]
x_test_text = largeText[size:]

y_train =  reviewTextProcessing['rating'][:size]
y_test = reviewTextProcessing['rating'][size:]
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)


In [169]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(100857, 33619, 100857, 33619)

In [170]:
#x_train_tokens[1], x_train_text[1]

In [171]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [172]:
np.mean(num_tokens), np.max(num_tokens)


(110.80553407299443, 4838)

In [173]:
np.std(num_tokens)

132.92503796578936

In [174]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


376

In [175]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.958706386269669

In [176]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape
x_test_pad.shape


(33619, 376)

In [177]:
for i in range(0, 1):
    print(x_train_pad[i])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [178]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [179]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [180]:
x_train_text[1]


'these little magnets are really powerful for there size, i am using them to make secret compartments in custom made boxes, each one hols about of a pound'

In [181]:
tokens_to_string(x_train_tokens[1])


'these little magnets are really powerful for there size i am using them to make secret compartments in custom made boxes each one about of a pound'

In [182]:
from keras import Sequential
from keras.layers import Dense, Embedding, Dropout
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import GlobalMaxPooling1D
from keras.optimizers import Adam

In [183]:
import tensorflow as tf
tf.__version__


'1.13.2'

In [184]:
optimizer = Adam(lr=1e-3)


In [185]:
model_cnn_02 = Sequential()
e = Embedding(num_words, 200, input_length=max_tokens)
model_cnn_02.add(e)
model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_02.add(GlobalMaxPooling1D())
model_cnn_02.add(Dense(256, activation='relu'))
model_cnn_02.add(Dense(1, activation='sigmoid'))
model_cnn_02.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model_cnn_02.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 376, 200)          2000000   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 375, 100)          40100     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               25856     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 2,066,213
Trainable params: 2,066,213
Non-trainable params: 0
_________________________________________________________________


In [186]:
index = len(x_train_pad)
index

100857

In [187]:
%%time
model_cnn_02.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=74)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Train on 95814 samples, validate on 5043 samples
Epoch 1/3
95814/95814 [==============================] - 384s 4ms/step - loss: 0.1907 - acc: 0.9335 - val_loss: 0.1572 - val_acc: 0.9431
Epoch 2/3
95814/95814 [==============================] - 382s 4ms/step - loss: 0.1278 - acc: 0.9507 - val_loss: 0.1603 - val_acc: 0.9467
Epoch 3/3
95814/95814 [==============================] - 379s 4ms/step - loss: 0.0902 - acc: 0.9653 - val_loss: 0.1738 - val_acc: 0.9435
Wall time: 19min 5s


In [188]:
%%time
result = model_cnn_02.evaluate(x_test_pad, y_test)

33619/33619 [==============================] - 49s 1ms/step
Wall time: 48.5 s


In [67]:
print("Accuracy: {0:.2%}".format(result[1]))


Accuracy: 95.01%


In [189]:
%%time
y_pred = model_cnn_02.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]


Wall time: 1.31 s


In [190]:
pd.DataFrame(y_pred).describe()

,0
count,1000.000000
mean,0.932637
std,0.182945
min,0.011366
25%,0.979855
50%,0.998276
75%,0.999831
max,1.000000


In [ ]:
#y_pred = y_pred.T[0]
#list(np.where(y_pred.T[0]))

In [ ]:
#y_pred

In [191]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])


In [192]:
cls_true = np.array(y_test[0:1000])
#cls_true = np.where(y_test[0:1000])
#cls_true

In [193]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]
incorrect

array([ 14,  39,  85, 133, 148, 155, 190, 191, 197, 204, 239, 365, 445,
       459, 475, 518, 534, 537, 542, 588, 616, 641, 658, 661, 663, 675,
       684, 723, 737, 758, 760, 762, 763, 790, 795, 877, 886, 914, 922,
       935, 944, 949, 957, 965, 970, 979, 983, 992], dtype=int64)

In [194]:
correct = np.where(cls_pred == cls_true)
#correct[0]

In [195]:
len(incorrect)


48

In [140]:
#np.where(y_pred.T)[0]

In [197]:
result_location = np.where(y_pred.T)[0]  + index
CNNresult = reviewTextProcessing.loc[list(result_location),  ['reviewText', 'overall', 'rating', 'Sentiment' , 'SentimentModel']]
#CNNresult.head(2)
#CNNresult.shape
#result_location
#result_location
CNNresult.head(5)

,reviewText,overall,rating,Sentiment,SentimentModel
100857,"Hunter 90434 16-Inch Portable Stand Fan, Oil R...",5.0,1,"[0.0, 0.78, 0.0, 0.0, 0.13636363636363635, 0.0...","[1.8063636363636364, 1.0]"
100858,Needed these because I was refinishing a large...,5.0,1,"[0.21428571428571427, 0.10571428571428572, 0.0...","[0.29833333333333334, 1.0]"
100859,"These are large and long lasting tack cloths, ...",5.0,1,[0.15285714285714286],"[0.15285714285714286, 1.0]"
100860,I need these all the time mfor my woodworking ...,5.0,1,"[0.0, 0.65]","[0.65, 1.0]"
100861,Good quality. They are packaged well and can ...,4.0,1,"[0.7, 0.0, 0.1]","[0.7999999999999999, 1.0]"


In [198]:
CNNresult['CNN_Sentiment'] = cls_pred


In [199]:
sumSentiment = lambda x: x[1]

In [200]:
CNNresult.columns = ['TextBlobAnalysis' if x=='Sentiment' else x for x in CNNresult.columns]
CNNresult.columns = ['TextBlob_Sentiment' if x=='SentimentModel' else x for x in CNNresult.columns]
CNNresult['TextBlob_Sentiment'] = CNNresult['TextBlob_Sentiment'].apply(sumSentiment)

In [201]:
CNNresult.CNN_Sentiment = CNNresult.CNN_Sentiment.astype(int)
CNNresult.TextBlob_Sentiment = CNNresult.TextBlob_Sentiment.astype(int)

In [208]:
CNNresult.loc[CNNresult.overall >= 3.0 , :].to_csv("Rating3.0above_CNN.csv")
CNNresult.loc[CNNresult.overall < 3.0 , :].to_csv("RatingBelow3.0_CNN.csv")

In [204]:
from sklearn.metrics import confusion_matrix

y_true = CNNresult.rating.to_list()
cnn_pred = CNNresult.CNN_Sentiment.to_list()
textBlob_pred = CNNresult.TextBlob_Sentiment.to_list()


In [205]:
#textBlob_pred

In [206]:
tn, fp, fn, tp = confusion_matrix(y_true, cnn_pred).ravel()
tn, fp, fn, tp

(31, 30, 18, 921)

In [207]:
tn, fp, fn, tp = confusion_matrix(y_true, textBlob_pred).ravel()
tn, fp, fn, tp

(21, 40, 93, 846)

In [ ]:
idx = incorrect[1]
print(type(idx))
for each in incorrect[:10]:
    idx = each
    text = x_test_text[idx]
    print(idx, text, cls_pred[idx], cls_true[idx], " BlobText ",
         TextBlob(text).sentiment.polarity)


In [ ]:
pd.options.display.max_colwidth = 500

#productsReviewData.cleanedText.head(2)